# Amazon Bedrock AgentCore Runtime 上の TypeScript MCP サーバーを使用した MCP クライアントのテスト

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime 環境を使用して TypeScript ベースの MCP (Model Context Protocol) サーバーをホストする方法を学びます。

### チュートリアル詳細

| 情報                | 詳細                                                      |
|:--------------------|:----------------------------------------------------------|
| チュートリアルタイプ | TypeScript MCP サーバーのホスティング                     |
| ツールタイプ        | MCP サーバー                                              |
| チュートリアルコンポーネント | AgentCore Runtime 上での TypeScript MCP サーバーホスティング |
| チュートリアル分野  | クロスバーティカル                                        |
| 難易度              | 簡単                                                      |
| 使用 SDK            | Anthropic の MCP 用 TypeScript SDK                        |


### チュートリアル概要

1. AgentCore Runtime の認証では、Amazon Cognito を使用してデプロイされた MCP サーバーにアクセスするための JWT トークンを提供します。

2. MCP サーバーは TypeScript で記述され、[カスタムフローを使用してデプロイ](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html)されます。

3. MCP クライアントは Python で記述されています。
   _注: MCP クライアントは任意の言語で記述できます。_

## 前提条件

このチュートリアルを実行するには以下が必要です：
- Node.js v22 以降 (MCP サーバー)
- Python 3.10+ (MCP クライアント)
- Docker (コンテナ化用)
- Amazon ECR (Docker イメージ保存用の Elastic Container Registry)
- Bedrock AgentCore にアクセス可能な AWS アカウント
- MCP (Model Context Protocol) ライブラリ
- Docker が実行中であること

In [ ]:
#!uv add -r requirements.txt --active

## MCP (Model Context Protocol) の理解

MCP は AI モデルが外部データやツールに安全にアクセスできるようにするプロトコルです。主なコンセプト：

* **ツール**: AI が呼び出してアクションを実行できる関数
* **プロンプト**: サーバーが LLM とのインタラクション用に構造化されたメッセージと指示を提供できるようにする
* **Streamable HTTP**: AgentCore Runtime が使用するトランスポートプロトコル
* **セッション分離**: 各クライアントは `Mcp-Session-Id` ヘッダーを通じて分離されたセッションを取得
* **ステートレス操作**: スケーラビリティのためにサーバーはステートレス操作をサポートする必要がある

AgentCore Runtime は MCP サーバーがデフォルトパスとして `0.0.0.0:8000/mcp` でホストされることを期待しています。

## ステップ 1: 認証用の Amazon Cognito のセットアップ

AgentCore Runtime には認証が必要です。デプロイされた MCP サーバーにアクセスするための JWT トークンを提供するために Amazon Cognito を使用します。

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role, setup_cognito_user_pool

In [ ]:
print("Amazon Cognito ユーザープールをセットアップ中...")
cognito_config = setup_cognito_user_pool()
print("Cognito セットアップ完了 ✓")
print(f"ユーザープール ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"クライアント ID: {cognito_config.get('client_id', 'N/A')}")

## ステップ 2: IAM 実行ロールの作成

開始する前に、AgentCore Runtime 用の IAM ロールを作成しましょう。このロールは Runtime の操作に必要な権限を提供します。

In [ ]:
tool_name = "mcp_server_ac"
print(f"{tool_name} 用の IAM ロールを作成中...")
agentcore_iam_role = create_agentcore_role(agent_name=tool_name)
print(f"IAM ロール作成完了 ✓")
print(f"ロール ARN: {agentcore_iam_role['Role']['Arn']}")

## ステップ 3: MCP サーバーの作成

2つのシンプルなツールと1つのプロンプトを持つ TypeScript MCP サーバーを作成しましょう。このチュートリアルの src フォルダに移動します。

1. 依存関係のインストール

```
npm install
```

2. AWS 認証情報のセットアップ
```
aws configure
export AWS_ACCESS_KEY_ID=your_access_key
export AWS_SECRET_ACCESS_KEY=your_secret_key
export AWS_REGION=us-east-1
```

3. サーバーの起動（ローカル実行）
```
npm run start
```

## ステップ 4: Docker を使用した MCP サーバーのデプロイ

注意: これはスターターツールキットを使用せずにエージェントまたは MCP サーバーをデプロイするための手動ステップです

https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

1. ECR リポジトリの作成
```
aws ecr create-repository --repository-name mcp-server --region us-east-1
```
2. イメージをビルドして ECR にプッシュ
```
# ログイントークンを取得
aws ecr get-login-password --region us-east-1 | \
  docker login --username AWS --password-stdin [account-id].dkr.ecr.us-east-1.amazonaws.com

docker buildx --platform linux/arm64 \
  -t [account-id].dkr.ecr.us-east-1.amazonaws.com/mcp-server:latest --push .
```

3. Bedrock AgentCore へのデプロイ

    - AWS コンソール → Bedrock → AgentCore → Create Agent に移動
    - プロトコルとして MCP を選択
    - Agent Runtime を設定:
        - Image URI: [account-id].dkr.ecr.us-east-1.amazonaws.com/mcp-server:latest
        - Bedrock モデルアクセス用の IAM 権限を設定
        - デプロイして Agent Sandbox でテスト
    - Discovery url: 上記から URL を選択、cognito_config['discovery_url']
    - Client id: 上記からクライアント ID を選択、cognito_config['client_id']
    - 実行ロール: 上記から ARN を選択、agentcore_iam_role['Role']['Arn']

## ステップ 5: リモートアクセス用の設定の保存

デプロイされた MCP サーバーを呼び出す前に、Agent ARN（ステップ 4 から ARN を取得）と Cognito 設定を AWS Systems Manager Parameter Store と AWS Secrets Manager に保存して、簡単に取得できるようにしましょう：

In [ ]:
import boto3
import json

boto_session = Session()
region = boto_session.region_name

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito 認証情報を Secrets Manager に保存しました")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Secrets Manager の Cognito 認証情報を更新しました")

 # 注意: ステップ 4 で作成したエージェント ARN を追加してください
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value="Add your agent arn that you created in step 4", 
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ エージェント ARN を Parameter Store に保存しました")

## ステップ 6: リモートテストクライアントの作成

デプロイされた MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイされたサーバーに接続します：

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"使用する AWS リージョン: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"エージェント ARN を取得: {agent_arn}")
     
        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Secrets Manager からベアラートークンを取得しました")
        
    except Exception as e:
        print(f"認証情報の取得エラー: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("エラー: BEARER_TOKEN が正しく取得できませんでした")
        sys.exit(1)
    

    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\n接続先: {mcp_url}")
    print("ヘッダー設定完了 ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCP セッションを初期化中...")
                await session.initialize()
                print("✓ MCP セッション初期化完了")
                
                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()
                
                print("\n📋 利用可能な MCP ツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   説明: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   パラメータ: {list(properties.keys())}")
                    print()
                
                print(f"✅ MCP サーバーへの接続成功!")
                print(f"{len(tool_result.tools)} 個のツールが利用可能です。")
                
    except Exception as e:
        print(f"❌ MCP サーバーへの接続エラー: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## ステップ 7: デプロイされた MCP サーバーのテスト

リモートクライアントを使用してデプロイされた MCP サーバーをテストしましょう：

In [ ]:
print("デプロイされた MCP サーバーをテスト中...")
print("=" * 50)
!python my_mcp_client_remote.py

## ステップ 8: MCP ツールのリモート呼び出し

ツールをリストするだけでなく、完全な MCP 機能を示すために実際に呼び出す拡張クライアントを作成しましょう：

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"使用する AWS リージョン: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"エージェント ARN を取得: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Secrets Manager からベアラートークンを取得しました")
        
    except Exception as e:
        print(f"認証情報の取得エラー: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\n接続先: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 MCP セッションを初期化中...")
                await session.initialize()
                print("✓ MCP セッション初期化完了")
                
                print("\n🔄 利用可能なツールを一覧表示中...")
                tool_result = await session.list_tools()
                
                print("\n📋 利用可能な MCP ツール:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 MCP ツールをテスト中:")
                print("=" * 50)
                
                try:
                    print("\n➕ add(5, 3) をテスト中...")
                    add_result = await session.call_tool(
                        name="add",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   結果: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")
                
                try:
                    print("\n✖️  subtract(10, 2) をテスト中...")
                    substract_result = await session.call_tool(
                        name="subtract",
                        arguments={"a": 10, "b": 2}
                    )
                    print(f"   結果: {substract_result.content[0].text}")
                except Exception as e:
                    print(f"   エラー: {e}")
                
                print("\n✅ MCP ツールテスト完了!")
                
    except Exception as e:
        print(f"❌ MCP サーバーへの接続エラー: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## ツール呼び出しのテスト

実際に MCP ツールを呼び出してテストしましょう：

In [ ]:
print("MCP ツール呼び出しをテスト中...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

MCP サーバーを AgentCore Runtime に正常にデプロイしたので、以下を行うことができます：

1. **ツールの追加**: MCP サーバーに追加のツールを拡張
2. **カスタム認証**: カスタム JWT オーソライザーの実装
3. **統合**: 他の AgentCore サービスとの統合

# 🎉 おめでとうございます！

以下を正常に完了しました：

✅ **TypeScript MCP サーバーの作成** - カスタムツール付き
✅ **認証のセットアップ** - Amazon Cognito を使用
✅ **AWS へのデプロイ** - AgentCore Runtime を使用
✅ **リモート呼び出し** - 適切な認証で
✅ **MCP コンセプトの学習** - ベストプラクティス付き

MCP サーバーは Amazon Bedrock AgentCore Runtime で実行されており、本番環境で使用する準備ができています！